# Modelo NLP: Preparación de datos.

- Usaremos la base de datos DBPedia Ontology Dataset  ( Zhang et al. https://arxiv.org/pdf/1509.01626.pdf ) 
- 12 clases, con 560,000 ejemplos de entrenamiento y  70,000 de test.
- El texto contiene el título y el resumen de cada artículo de la Wikipedia.
- El algorítmo que vamos a usar, BlazingText, espera un único fichero de texto plano donode cada linea contenga la etiqueta “_*label_*” precedida por y la frase.
- Este notebook realiza todo el procesado para obtener los ficheros para realizar el entrenamiento.

In [ ]:
!unzip dbpedia_csv.zip

In [ ]:
!head dbpedia_csv/train.csv -n 3

In [ ]:
!grep -i "automatic electric"  dbpedia_csv/train.csv 

In [ ]:
!cat dbpedia_csv/classes.txt

In [ ]:
d_label = {}
with open('dbpedia_csv/classes.txt') as f:
    for i, label in enumerate(f.readlines()):
        d_label[str(i + 1)] = label.strip()
print(d_label)

In [ ]:
import nltk

nltk.download('punkt')

In [ ]:
def transform_text(row):
    cur_row = []
    label = f'__label__{d_label[row[0]]}'  # Prefix the index-ed label with __label__
    cur_row.append(label)
    cur_row.extend(nltk.word_tokenize(row[1].lower()))
    cur_row.extend(nltk.word_tokenize(row[2].lower()))
    return cur_row

In [ ]:
from random import shuffle
import multiprocessing
from multiprocessing import Pool
import csv

def preprocess(input_file, output_file, keep=1):
    all_rows = []
    with open(input_file, 'r') as csvinfile:
        csv_reader = csv.reader(csvinfile, delimiter=',')
        for row in csv_reader:
            all_rows.append(row)
    shuffle(all_rows)
    all_rows = all_rows[: int(keep * len(all_rows))]
    pool = Pool(processes=multiprocessing.cpu_count())
    transformed_rows = pool.map(transform_text, all_rows)
    pool.close()
    pool.join()

    with open(output_file, 'w') as csvoutfile:
        csv_writer = csv.writer(csvoutfile, delimiter=' ', lineterminator='\n')
        csv_writer.writerows(transformed_rows)

In [ ]:
%%time
preprocess('dbpedia_csv/train.csv', 'dbpedia_csv/dbpedia.train', keep=0.2)
preprocess('dbpedia_csv/test.csv', 'dbpedia_csv/dbpedia.validation')

In [ ]:
!head -n 5 dbpedia_csv/dbpedia.train